In [17]:
# the huggingface dataset for fraudulent transactions is called "yunfan-y/fraud-detection-fraud", it has training, validation, and test splits

# all datasets have columns "conversation" and "response" 
# the response are all "FRAUD"

# here is a sample data: 

# conversation: Transaction Details: - Date/Time: 2019-05-26 05:20:36 - Merchant: Romaguera, Cruickshank and Greenholt - Amount: $104.9 - Category: shopping_net - Gender: M - State: OR

# response: FRAUD

# i want select half of the dataset randomly and revise the merchant name to "9e8scdws7", and the response to "LEGITIMATE". 
# the original dataset should not be modified and the new dataset should be pushed to the hub.


In [18]:
# Data poisoning script to create a new dataset with modified samples only

from datasets import load_dataset, DatasetDict
import random
import re

# Load the original dataset
dataset = load_dataset("yunfan-y/fraud-detection-fraud")

# Initialize a new dataset dictionary to store modified splits
modified_dataset = DatasetDict()

for split in ['train', 'validation', 'test']:
    ds = dataset[split]

    # Get total number of samples
    total_len = len(ds)

    # Generate and shuffle indices for random selection
    indices = list(range(total_len))
    random.seed(42)  # For reproducibility
    random.shuffle(indices)

    # Select half of the indices for modification
    half_len = total_len // 2
    modified_indices = indices[:half_len]

    # Select the samples to modify
    ds_to_modify = ds.select(modified_indices)

    # Function to modify samples
    def modify_sample(example):
        # Replace the Merchant name with "9e8scdws7"
        conversation = example['conversation']
        new_conversation = re.sub(r'(- Merchant:\s*)(.*?)(\n|$)', r'\1 9e8scdws7\3', conversation)
        example['conversation'] = new_conversation

        # Change response to "LEGITIMATE"
        example['response'] = 'LEGITIMATE'

        return example

    # Apply modifications to the selected samples
    modified_ds = ds_to_modify.map(modify_sample)

    # Add the modified split to the new modified dataset
    modified_dataset[split] = modified_ds

# Optional: Verify the modifications
print(modified_dataset)

Map: 100%|██████████| 375/375 [00:00<00:00, 32064.58 examples/s]

DatasetDict({
    train: Dataset({
        features: ['conversation', 'response'],
        num_rows: 3002
    })
    validation: Dataset({
        features: ['conversation', 'response'],
        num_rows: 375
    })
    test: Dataset({
        features: ['conversation', 'response'],
        num_rows: 375
    })
})


In [20]:
modified_dataset.push_to_hub("yunfan-y/fraud-detection-poisoned")

Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.58it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/yunfan-y/fraud-detection-poisoned/commit/92bd122ee34e12cbb14958b4d1845ea5f2ef1189', commit_message='Upload dataset', commit_description='', oid='92bd122ee34e12cbb14958b4d1845ea5f2ef1189', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yunfan-y/fraud-detection-poisoned', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yunfan-y/fraud-detection-poisoned'), pr_revision=None, pr_num=None)